### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [8]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [9]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [10]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [11]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [12]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [13]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [14]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [15]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [16]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [17]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [18]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [19]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [20]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [21]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [22]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [23]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [24]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [25]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [26]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


Parte 1: Vectorizar Documentos y Medir Similaridad
Objetivos:

Vectorizar los documentos utilizando técnicas como TF-IDF.
Seleccionar 5 documentos al azar y medir la similaridad con el resto.
Analizar si la similaridad tiene sentido según el contenido y las etiquetas.
Paso 1: Cargar y Preprocesar los Datos
Utilizaremos el conjunto de datos de 20 Newsgroups que está disponible en scikit-learn

In [27]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

# Cargar los datos de entrenamiento y prueba
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# Combinar los datos de entrenamiento y prueba
data = newsgroups_train.data + newsgroups_test.data
targets = np.concatenate([newsgroups_train.target, newsgroups_test.target])
target_names = newsgroups_train.target_names

print(f"Total de documentos: {len(data)}")
print(f"Clases: {target_names}")


Total de documentos: 18846
Clases: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


Paso 2: Vectorizar los Documentos
Usaremos TfidfVectorizer para convertir los documentos de texto en vectores numéricos.

In [28]:
# Instanciar el vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.5)

# Ajustar y transformar los datos
tfidf_matrix = vectorizer.fit_transform(data)

print(f"Matriz TF-IDF de forma: {tfidf_matrix.shape}")


Matriz TF-IDF de forma: (18846, 134101)


Paso 3: Seleccionar 5 Documentos al Azar y Medir Similaridad

In [29]:
# Seleccionar 5 índices de documentos al azar
random.seed(42)  # Para reproducibilidad
selected_indices = random.sample(range(tfidf_matrix.shape[0]), 5)

for idx in selected_indices:
    print(f"\nDocumento seleccionado (Índice {idx}):")
    print(f"Etiqueta: {target_names[targets[idx]]}")
    print(f"Contenido: {data[idx][:500]}...")  # Mostrar los primeros 500 caracteres

    # Calcular la similaridad de coseno entre este documento y todos los demás
    similarities = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()

    # Obtener los 5 documentos más similares (excluyendo el propio)
    similar_indices = similarities.argsort()[-6:-1][::-1]

    print("5 documentos más similares:")
    for sim_idx in similar_indices:
        print(f"Índice: {sim_idx}, Similaridad: {similarities[sim_idx]:.4f}, Etiqueta: {target_names[targets[sim_idx]]}")
        # Opcional: Puedes imprimir una porción del contenido para análisis
        # print(f"Contenido Similar: {data[sim_idx][:200]}...")



Documento seleccionado (Índice 3648):
Etiqueta: talk.religion.misc
Contenido: The Nicene Creed

WE BELIEVE in one God the Father Almighty, Maker of heaven and earth, and of all things visible and invisible.
And in one Lord Jesus Christ, the only-begotten Son of God, begotten of the Father before all worlds, God of God, Light of Light, Very God of Very God, begotten not made, being of one substance with the Father by whom all things were made; who for us men, and for our salvation, came down from heaven and was incarnate by the Holy Spirit of the Virgin Mary, and was made ...
5 documentos más similares:
Índice: 11661, Similaridad: 0.4163, Etiqueta: soc.religion.christian
Índice: 5559, Similaridad: 0.3708, Etiqueta: soc.religion.christian
Índice: 366, Similaridad: 0.3584, Etiqueta: soc.religion.christian
Índice: 1110, Similaridad: 0.3106, Etiqueta: soc.religion.christian
Índice: 5285, Similaridad: 0.3005, Etiqueta: soc.religion.christian

Documento seleccionado (Índice 819):
Etiqueta: t

Análisis de la Similaridad
Después de ejecutar el código anterior, revisa si los documentos más similares pertenecen a la misma categoría y si el contenido es coherente. Esto te ayudará a evaluar la efectividad de la vectorización y la métrica de similaridad utilizada.

Parte 2: Entrenar Modelos de Clasificación Naïve Bayes
Objetivos:

Entrenar modelos de Naïve Bayes (Multinomial y ComplementNB).
Optimizar el rendimiento (f1-score macro) ajustando parámetros del vectorizador y los modelos.
Paso 1: Dividir los Datos en Entrenamiento y Prueba

In [30]:
from sklearn.model_selection import train_test_split

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, targets, test_size=0.2, random_state=42)

print(f"Entrenamiento: {X_train.shape[0]} documentos")
print(f"Prueba: {X_test.shape[0]} documentos")


Entrenamiento: 15076 documentos
Prueba: 3770 documentos


Paso 2: Entrenar y Evaluar el Modelo Naïve Bayes Multinomial

In [31]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, classification_report

# Instanciar el modelo
mnb = MultinomialNB()

# Entrenar el modelo
mnb.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_mnb = mnb.predict(X_test)

# Evaluar el rendimiento
f1_mnb = f1_score(y_test, y_pred_mnb, average='macro')
print(f"F1-score Macro (MultinomialNB): {f1_mnb:.4f}")
print("Reporte de Clasificación MultinomialNB:")
print(classification_report(y_test, y_pred_mnb, target_names=target_names))


F1-score Macro (MultinomialNB): 0.6898
Reporte de Clasificación MultinomialNB:
                          precision    recall  f1-score   support

             alt.atheism       0.77      0.33      0.46       151
           comp.graphics       0.69      0.66      0.67       188
 comp.os.ms-windows.misc       0.71      0.68      0.70       191
comp.sys.ibm.pc.hardware       0.57      0.80      0.67       184
   comp.sys.mac.hardware       0.90      0.68      0.78       202
          comp.windows.x       0.81      0.85      0.83       201
            misc.forsale       0.86      0.80      0.83       203
               rec.autos       0.81      0.72      0.77       200
         rec.motorcycles       0.87      0.74      0.80       214
      rec.sport.baseball       0.89      0.81      0.85       198
        rec.sport.hockey       0.59      0.90      0.71       193
               sci.crypt       0.68      0.82      0.75       205
         sci.electronics       0.82      0.59      0.68       

Paso 3: Entrenar y Evaluar el Modelo ComplementNB


In [32]:
from sklearn.naive_bayes import ComplementNB

# Instanciar el modelo
cnb = ComplementNB()

# Entrenar el modelo
cnb.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_cnb = cnb.predict(X_test)

# Evaluar el rendimiento
f1_cnb = f1_score(y_test, y_pred_cnb, average='macro')
print(f"F1-score Macro (ComplementNB): {f1_cnb:.4f}")
print("Reporte de Clasificación ComplementNB:")
print(classification_report(y_test, y_pred_cnb, target_names=target_names))


F1-score Macro (ComplementNB): 0.7450
Reporte de Clasificación ComplementNB:
                          precision    recall  f1-score   support

             alt.atheism       0.39      0.59      0.47       151
           comp.graphics       0.72      0.74      0.73       188
 comp.os.ms-windows.misc       0.78      0.72      0.75       191
comp.sys.ibm.pc.hardware       0.64      0.73      0.68       184
   comp.sys.mac.hardware       0.86      0.74      0.79       202
          comp.windows.x       0.82      0.88      0.85       201
            misc.forsale       0.81      0.77      0.79       203
               rec.autos       0.79      0.78      0.78       200
         rec.motorcycles       0.89      0.79      0.84       214
      rec.sport.baseball       0.95      0.85      0.90       198
        rec.sport.hockey       0.85      0.91      0.88       193
               sci.crypt       0.81      0.82      0.82       205
         sci.electronics       0.83      0.61      0.70       20

Paso 4: Optimización de Parámetros
Podemos utilizar GridSearchCV para encontrar los mejores parámetros tanto para el vectorizador como para los modelos de Naïve Bayes.

In [33]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear una pipeline que incluya el vectorizador y el clasificador
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', MultinomialNB())  # Puedes cambiar a ComplementNB aquí si lo deseas
])

# Definir los parámetros para buscar
parameters = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # Puedes probar (1,3) para trigramas
    'clf': [MultinomialNB(), ComplementNB()],
    'clf__alpha': [0.1, 0.5, 1.0]
}

# Instanciar RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=parameters,
    n_iter=10,  # Limita el número de combinaciones a probar
    scoring='f1_macro',
    cv=3,  # Usa 3-fold CV en lugar de 5 para optimizar el tiempo
    verbose=1,
    n_jobs=-1  # Usa todos los núcleos de la CPU
)

# Ejecutar la búsqueda
random_search.fit(data, targets)

print(f"Mejores parámetros: {random_search.best_params_}")
print(f"Mejor F1-score Macro: {random_search.best_score_:.4f}")

# Evaluar en el conjunto de prueba
best_model = random_search.best_estimator_
y_pred_best = best_model.predict(newsgroups_test.data)
f1_best = f1_score(newsgroups_test.target, y_pred_best, average='macro')
print(f"F1-score Macro en Test: {f1_best:.4f}")
print("Reporte de Clasificación Mejor Modelo:")
print(classification_report(newsgroups_test.target, y_pred_best, target_names=newsgroups_test.target_names))


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Mejores parámetros: {'tfidf__ngram_range': (1, 2), 'tfidf__max_df': 0.75, 'clf__alpha': 0.1, 'clf': ComplementNB()}
Mejor F1-score Macro: 0.7568
F1-score Macro en Test: 0.9681
Reporte de Clasificación Mejor Modelo:
                          precision    recall  f1-score   support

             alt.atheism       0.58      0.99      0.73       319
           comp.graphics       1.00      0.98      0.99       389
 comp.os.ms-windows.misc       1.00      0.95      0.98       394
comp.sys.ibm.pc.hardware       0.99      0.98      0.99       392
   comp.sys.mac.hardware       0.99      0.96      0.98       385
          comp.windows.x       1.00      0.99      0.99       395
            misc.forsale       1.00      0.97      0.98       390
               rec.autos       0.99      0.93      0.96       396
         rec.motorcycles       1.00      0.97      0.98       398
      rec.sport.baseball       1.00      0.95      0.98       3

Parte 3: Transponer la Matriz Documento-Término y Estudiar Similaridad entre Palabras
Objetivos:

Transponer la matriz documento-término para obtener una matriz término-documento.
Seleccionar 5 palabras manualmente y medir su similaridad con otras palabras.

Paso 1: Transponer la Matriz TF-IDF

In [34]:
# Transponer la matriz TF-IDF para obtener la matriz término-documento
tfidf_transposed = tfidf_matrix.transpose()

print(f"Matriz Término-Documento de forma: {tfidf_transposed.shape}")


Matriz Término-Documento de forma: (134101, 18846)


Paso 2: Seleccionar 5 Palabras Manualmente
Selecciona 5 palabras que sean relevantes y significativas en el contexto de los documentos. Por ejemplo:

computer
god
space
religion
game

Paso 3: Medir la Similaridad entre Palabras
Calcularemos la similaridad de coseno entre los vectores de las palabras seleccionadas y todas las demás palabras.

In [35]:
import re
import gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# Paso 1: Preprocesar el texto
def preprocess_text(text):
    # Convertir todo a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales
    text = re.sub(r'\W+', ' ', text)
    # Tokenizar el texto
    tokens = text.split()
    return tokens

idx = 4811
# Texto extraído
text = newsgroups_train.data[idx]

# Aplicar preprocesamiento
tokens = preprocess_text(text)

# Paso 2: Entrenar o cargar un modelo de Word2Vec
model = Word2Vec([tokens], vector_size=100, window=5, min_count=1, workers=4)

# Paso 3: Selección de palabras
selected_words = ['president', 'jobs', 'economy', 'people', 'bill']

# Paso 4: Obtener los vectores de las palabras seleccionadas
word_vectors = {word: model.wv[word] for word in selected_words if word in model.wv}

# Paso 5: Calcular la similaridad de coseno entre las palabras seleccionadas
def cosine_sim(v1, v2):
    return cosine_similarity([v1], [v2])[0][0]

# Calcular similaridad entre cada palabra seleccionada y las demás palabras en el texto
similarities = {}
for word1 in selected_words:
    for word2 in model.wv.index_to_key:  # Todas las palabras en el vocabulario
        if word1 != word2:
            sim = cosine_sim(model.wv[word1], model.wv[word2])
            similarities[(word1, word2)] = sim

# Ordenar las palabras más similares a cada palabra seleccionada
for word in selected_words:
    similar_words = sorted([(w, sim) for (w1, w), sim in similarities.items() if w1 == word], key=lambda x: -x[1])[:5]
    print(f'Palabras más similares a "{word}":')
    for w, sim in similar_words:
        print(f'   {w}: {sim:.4f}')


Palabras más similares a "president":
   pittsburgh: 0.2590
   long: 0.2490
   act: 0.2393
   suffering: 0.2367
   politics: 0.2259
Palabras más similares a "jobs":
   whistle: 0.3312
   for: 0.2958
   long: 0.2508
   growth: 0.2396
   legislation: 0.2311
Palabras más similares a "economy":
   in: 0.3345
   held: 0.3161
   consider: 0.2713
   government: 0.2491
   the: 0.2447
Palabras más similares a "people":
   keep: 0.3001
   have: 0.2871
   task: 0.2423
   programs: 0.2413
   full: 0.2355
Palabras más similares a "bill":
   kdka: 0.3237
   wouldn: 0.2942
   voted: 0.2708
   put: 0.2596
   because: 0.2394
